In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Input, LSTM
from keras import preprocessing 
from keras.utils import pad_sequences

import os
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.getcwd()

'/content'

In [4]:
os.chdir('/content/drive/MyDrive/MedicalChatBot/Chatbot Creation')

In [5]:
os.getcwd()

'/content/drive/MyDrive/MedicalChatBot/Chatbot Creation'

In [6]:
file1 = open('Question & Answer.json')

Importing the File

In [7]:
df1 =json.load(file1)

In [8]:
df = df1[:2000]
print(len(df))

2000


In [9]:
questions=[]
short_answers=[]
answers=[]
for i in  range(len(df)):
    questions.append(df[i]['question'])
    short_answers.append(df[i]['short_answer'])
    answers.append(df[i]['answer'])



In [13]:
len(answers),len(df),len(short_answers),len(questions)

(2000, 2000, 2000, 2000)

# Adding Tags

In [14]:
short_answers1=[]
for i in short_answers:
    short_answers1.append(i.replace('\n',''))


In [15]:
short_answers1[:10]

['Dental implants',
 'Titanium is better',
 'Zirconium',
 'More research',
 'Not very common',
 'No advantages',
 'Sertraline',
 'Weight changes',
 '?bruise',
 'CAT BITE WOUND']

In [16]:
short_answers2=[]
for i in  short_answers1:
    short_answers2.append('<START>'+i+'<END>')



In [17]:
short_answers2[:10]

['<START>Dental implants<END>',
 '<START>Titanium is better<END>',
 '<START>Zirconium<END>',
 '<START>More research<END>',
 '<START>Not very common<END>',
 '<START>No advantages<END>',
 '<START>Sertraline<END>',
 '<START>Weight changes<END>',
 '<START>?bruise<END>',
 '<START>CAT BITE WOUND<END>']

In [18]:
[questions+short_answers2]

[['zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
  "zoloft/ sertaline cause heart racing can't sit still. is this normal?",
  'zoloft (sertraline) side effects? does i

In [19]:
tokenizer = preprocessing.text.Tokenizer()

In [20]:
texts =questions+short_answers2

In [21]:
tokenizer.fit_on_texts(texts)

In [22]:
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 3184


# Encoder Input Data

In [23]:
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

(2000, 58) 58


# Decoder input Data

In [28]:
tokenized_answers = tokenizer.texts_to_sequences(short_answers2)

maxlen_answers = max([len(x) for x in short_answers2])
padded_answers = pad_sequences(tokenized_answers,maxlen= maxlen_answers,padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape,maxlen_answers)

(2000, 49) 49


# Decoder Output Data

In [29]:
from keras.utils import to_categorical

In [30]:
tokenized_answers = tokenizer.texts_to_sequences(short_answers2)
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = pad_sequences(tokenized_answers, maxlen= maxlen_answers, padding ='post' )

one_hot_answers = to_categorical(padded_answers,VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)

(2000, 49, 3184)


# Model Building

In [31]:
from keras.layers import Embedding, Dense
import keras
from keras.models import Model

In [32]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(VOCAB_SIZE,200,mask_zero=True)(encoder_inputs)
encoder_output,state_h,state_c = LSTM(200,return_state= True)(encoder_embedding)

encoder_states = [state_h,state_c]

In [33]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(VOCAB_SIZE,200,mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(200,return_state= True,return_sequences = True)
decoder_outputs,_,_ = decoder_lstm(decoder_embedding, initial_state =encoder_states)
decoder_dense = Dense(VOCAB_SIZE,activation=keras.activations.softmax)

output = decoder_dense(decoder_outputs)



In [34]:
model = Model([encoder_inputs, decoder_inputs], output)

In [35]:
model.compile(optimizer=keras.optimizers.RMSprop(), loss='categorical_crossentropy',metrics=['accuracy'])


In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 200)    636800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 200)    636800      ['input_2[0][0]']                
                                                                                              

In [37]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'loss', mode= 'min',verbose=1, patience =50)
mc = ModelCheckpoint('MedicalChatbotModel.h5',monitor='accuracy',mode='max',verbose=1,save_best_only=True)

In [ ]:
model.fit([encoder_input_data, decoder_input_data],decoder_output_data,batch_size=50,epochs=200,callbacks=[es,mc],validation_split=0.2)


Epoch 1/200
32/32 [==============================] - ETA: 0s - loss: 0.0219 - accuracy: 0.8560
Epoch 1: accuracy improved from -inf to 0.85595, saving model to MedicalChatbotModel.h5
32/32 [==============================] - 26s 828ms/step - loss: 0.0219 - accuracy: 0.8560 - val_loss: 0.4683 - val_accuracy: 0.4839
Epoch 2/200
32/32 [==============================] - ETA: 0s - loss: 0.0216 - accuracy: 0.8511
Epoch 2: accuracy did not improve from 0.85595
32/32 [==============================] - 26s 812ms/step - loss: 0.0216 - accuracy: 0.8511 - val_loss: 0.4696 - val_accuracy: 0.4870
Epoch 3/200
32/32 [==============================] - ETA: 0s - loss: 0.0213 - accuracy: 0.8540
Epoch 3: accuracy did not improve from 0.85595
32/32 [==============================] - 25s 797ms/step - loss: 0.0213 - accuracy: 0.8540 - val_loss: 0.4669 - val_accuracy: 0.4846
Epoch 4/200
32/32 [==============================] - ETA: 0s - loss: 0.0214 - accuracy: 0.8534
Epoch 4: accuracy did not improve from 0.8

In [38]:
model_saved = keras.models.load_model('MedicalChatbotModel.h5')

# Inference Models

In [39]:
def inference_model():

    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(200,))
    decoder_state_input_c = Input(shape=(200,))

    decoder_state_inputs = [decoder_state_input_h,decoder_state_input_c]

    decoder_outputs,state_h,state_c = decoder_lstm(decoder_embedding,initial_state=decoder_state_inputs)
    decoder_states = [state_h,state_h]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs]+decoder_state_inputs,[decoder_outputs]+decoder_states)

    return encoder_model, decoder_model

In [40]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [41]:
enc_model , dec_model = inference_model()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : is aquaphor good for healing scabs
1/1 [==============================] - 0s 18ms/step
 else recovering purulent appeard sensitivity sprain becuase professional vaccinated gotten diabetes spit everyone pink inguinal discoloured style flap simple rule covered rare rule covered rare rule wind rule covered rare rule covered rare rule wind rule covered rare rule covered rare rule wind rule covered rare rule covered rare rule


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-e46c86910950>", line 4, in <module>
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelbase.py", line 863, in raw_input
    password=False,
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelbase.py", line 904, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During

KeyboardInterrupt: ignored